# ECOモデルの実装
本ファイルでは、ECOモデルを実装します。


In [13]:
%cd /content

/content


In [14]:
!git clone https://github.com/HisakaKoji/pytorch_advanced.git

Cloning into 'pytorch_advanced'...
remote: Enumerating objects: 52, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 495 (delta 26), reused 0 (delta 0), pack-reused 443
Receiving objects: 100% (495/495), 18.01 MiB | 38.26 MiB/s, done.
Resolving deltas: 100% (261/261), done.


In [15]:
%cd  pytorch_advanced/9_video_classification_eco/

/content/pytorch_advanced/9_video_classification_eco


# 9.2 学習目標

1.	ECOの2D Netモジュールの概要を理解する
2.	Inception-v2を実装できるようになる


# 9.3 学習目標

1.	ECOの3D Netモジュールの概要を理解する
2.	3D Resnetを実装できるようになる


# 事前準備

とくになし

In [0]:
import torch
from torch import nn

# ECOの2D Netモジュール

In [0]:
class BasicConv(nn.Module):
    '''ECOの2D Netモジュールの最初のモジュール'''

    def __init__(self):
        super(BasicConv, self).__init__()

        self.conv1_7x7_s2 = nn.Conv2d(3, 64, kernel_size=(
            7, 7), stride=(2, 2), padding=(3, 3))
        self.conv1_7x7_s2_bn = nn.BatchNorm2d(
            64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.conv1_relu_7x7 = nn.ReLU(inplace=True)
        self.pool1_3x3_s2 = nn.MaxPool2d(
            kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
        self.conv2_3x3_reduce = nn.Conv2d(
            64, 64, kernel_size=(1, 1), stride=(1, 1))
        self.conv2_3x3_reduce_bn = nn.BatchNorm2d(
            64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.conv2_relu_3x3_reduce = nn.ReLU(inplace=True)
        self.conv2_3x3 = nn.Conv2d(64, 192, kernel_size=(
            3, 3), stride=(1, 1), padding=(1, 1))
        self.conv2_3x3_bn = nn.BatchNorm2d(
            192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.conv2_relu_3x3 = nn.ReLU(inplace=True)
        self.pool2_3x3_s2 = nn.MaxPool2d(
            kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)

    def forward(self, x):
        out = self.conv1_7x7_s2(x)
        out = self.conv1_7x7_s2_bn(out)
        out = self.conv1_relu_7x7(out)
        out = self.pool1_3x3_s2(out)
        out = self.conv2_3x3_reduce(out)
        out = self.conv2_3x3_reduce_bn(out)
        out = self.conv2_relu_3x3_reduce(out)
        out = self.conv2_3x3(out)
        out = self.conv2_3x3_bn(out)
        out = self.conv2_relu_3x3(out)
        out = self.pool2_3x3_s2(out)
        return out


In [0]:
class InceptionA(nn.Module):
    '''InceptionA'''

    def __init__(self):
        super(InceptionA, self).__init__()

        self.inception_3a_1x1 = nn.Conv2d(
            192, 64, kernel_size=(1, 1), stride=(1, 1))
        self.inception_3a_1x1_bn = nn.BatchNorm2d(
            64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.inception_3a_relu_1x1 = nn.ReLU(inplace=True)

        self.inception_3a_3x3_reduce = nn.Conv2d(
            192, 64, kernel_size=(1, 1), stride=(1, 1))
        self.inception_3a_3x3_reduce_bn = nn.BatchNorm2d(
            64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.inception_3a_relu_3x3_reduce = nn.ReLU(inplace=True)
        self.inception_3a_3x3 = nn.Conv2d(
            64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.inception_3a_3x3_bn = nn.BatchNorm2d(
            64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.inception_3a_relu_3x3 = nn.ReLU(inplace=True)

        self.inception_3a_double_3x3_reduce = nn.Conv2d(
            192, 64, kernel_size=(1, 1), stride=(1, 1))
        self.inception_3a_double_3x3_reduce_bn = nn.BatchNorm2d(
            64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.inception_3a_relu_double_3x3_reduce = nn.ReLU(inplace=True)
        self.inception_3a_double_3x3_1 = nn.Conv2d(
            64, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.inception_3a_double_3x3_1_bn = nn.BatchNorm2d(
            96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.inception_3a_relu_double_3x3_1 = nn.ReLU(inplace=True)
        self.inception_3a_double_3x3_2 = nn.Conv2d(
            96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.inception_3a_double_3x3_2_bn = nn.BatchNorm2d(
            96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.inception_3a_relu_double_3x3_2 = nn.ReLU(inplace=True)

        self.inception_3a_pool = nn.AvgPool2d(
            kernel_size=3, stride=1, padding=1)
        self.inception_3a_pool_proj = nn.Conv2d(
            192, 32, kernel_size=(1, 1), stride=(1, 1))
        self.inception_3a_pool_proj_bn = nn.BatchNorm2d(
            32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.inception_3a_relu_pool_proj = nn.ReLU(inplace=True)

    def forward(self, x):

        out1 = self.inception_3a_1x1(x)
        out1 = self.inception_3a_1x1_bn(out1)
        out1 = self.inception_3a_relu_1x1(out1)

        out2 = self.inception_3a_3x3_reduce(x)
        out2 = self.inception_3a_3x3_reduce_bn(out2)
        out2 = self.inception_3a_relu_3x3_reduce(out2)
        out2 = self.inception_3a_3x3(out2)
        out2 = self.inception_3a_3x3_bn(out2)
        out2 = self.inception_3a_relu_3x3(out2)

        out3 = self.inception_3a_double_3x3_reduce(x)
        out3 = self.inception_3a_double_3x3_reduce_bn(out3)
        out3 = self.inception_3a_relu_double_3x3_reduce(out3)
        out3 = self.inception_3a_double_3x3_1(out3)
        out3 = self.inception_3a_double_3x3_1_bn(out3)
        out3 = self.inception_3a_relu_double_3x3_1(out3)
        out3 = self.inception_3a_double_3x3_2(out3)
        out3 = self.inception_3a_double_3x3_2_bn(out3)
        out3 = self.inception_3a_relu_double_3x3_2(out3)

        out4 = self.inception_3a_pool(x)
        out4 = self.inception_3a_pool_proj(out4)
        out4 = self.inception_3a_pool_proj_bn(out4)
        out4 = self.inception_3a_relu_pool_proj(out4)

        outputs = [out1, out2, out3, out4]

        return torch.cat(outputs, 1)


In [0]:
class InceptionB(nn.Module):
    '''InceptionB'''

    def __init__(self):
        super(InceptionB, self).__init__()
        
        self.inception_3b_1x1 = nn.Conv2d(
            256, 64, kernel_size=(1, 1), stride=(1, 1))
        self.inception_3b_1x1_bn = nn.BatchNorm2d(
            64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.inception_3b_relu_1x1 = nn.ReLU(inplace=True)

        self.inception_3b_3x3_reduce = nn.Conv2d(
            256, 64, kernel_size=(1, 1), stride=(1, 1))
        self.inception_3b_3x3_reduce_bn = nn.BatchNorm2d(
            64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.inception_3b_relu_3x3_reduce = nn.ReLU(inplace=True)
        self.inception_3b_3x3 = nn.Conv2d(
            64, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.inception_3b_3x3_bn = nn.BatchNorm2d(
            96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.inception_3b_relu_3x3 = nn.ReLU(inplace=True)

        self.inception_3b_double_3x3_reduce = nn.Conv2d(
            256, 64, kernel_size=(1, 1), stride=(1, 1))
        self.inception_3b_double_3x3_reduce_bn = nn.BatchNorm2d(
            64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.inception_3b_relu_double_3x3_reduce = nn.ReLU(inplace=True)
        self.inception_3b_double_3x3_1 = nn.Conv2d(
            64, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.inception_3b_double_3x3_1_bn = nn.BatchNorm2d(
            96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.inception_3b_relu_double_3x3_1 = nn.ReLU(inplace=True)
        self.inception_3b_double_3x3_2 = nn.Conv2d(
            96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.inception_3b_double_3x3_2_bn = nn.BatchNorm2d(
            96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.inception_3b_relu_double_3x3_2 = nn.ReLU(inplace=True)

        self.inception_3b_pool = nn.AvgPool2d(
            kernel_size=3, stride=1, padding=1)
        self.inception_3b_pool_proj = nn.Conv2d(
            256, 64, kernel_size=(1, 1), stride=(1, 1))
        self.inception_3b_pool_proj_bn = nn.BatchNorm2d(
            64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.inception_3b_relu_pool_proj = nn.ReLU(inplace=True)

    def forward(self, x):
        
        out1 = self.inception_3b_1x1(x)
        out1 = self.inception_3b_1x1_bn(out1)
        out1 = self.inception_3b_relu_1x1(out1)

        out2 = self.inception_3b_3x3_reduce(x)
        out2 = self.inception_3b_3x3_reduce_bn(out2)
        out2 = self.inception_3b_relu_3x3_reduce(out2)
        out2 = self.inception_3b_3x3(out2)
        out2 = self.inception_3b_3x3_bn(out2)
        out2 = self.inception_3b_relu_3x3(out2)

        out3 = self.inception_3b_double_3x3_reduce(x)
        out3 = self.inception_3b_double_3x3_reduce_bn(out3)
        out3 = self.inception_3b_relu_double_3x3_reduce(out3)
        out3 = self.inception_3b_double_3x3_1(out3)
        out3 = self.inception_3b_double_3x3_1_bn(out3)
        out3 = self.inception_3b_relu_double_3x3_1(out3)
        out3 = self.inception_3b_double_3x3_2(out3)
        out3 = self.inception_3b_double_3x3_2_bn(out3)
        out3 = self.inception_3b_relu_double_3x3_2(out3)

        out4 = self.inception_3b_pool(x)
        out4 = self.inception_3b_pool_proj(out4)
        out4 = self.inception_3b_pool_proj_bn(out4)
        out4 = self.inception_3b_relu_pool_proj(out4)

        outputs = [out1, out2, out3, out4]

        return torch.cat(outputs, 1)


In [0]:
class InceptionC(nn.Module):
    '''InceptionC'''

    def __init__(self):
        super(InceptionC, self).__init__()

        self.inception_3c_double_3x3_reduce = nn.Conv2d(
            320, 64, kernel_size=(1, 1), stride=(1, 1))
        self.inception_3c_double_3x3_reduce_bn = nn.BatchNorm2d(
            64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.inception_3c_relu_double_3x3_reduce = nn.ReLU(inplace=True)
        self.inception_3c_double_3x3_1 = nn.Conv2d(
            64, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.inception_3c_double_3x3_1_bn = nn.BatchNorm2d(
            96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.inception_3c_relu_double_3x3_1 = nn.ReLU(inplace=True)

    def forward(self, x):
        out = self.inception_3c_double_3x3_reduce(x)
        out = self.inception_3c_double_3x3_reduce_bn(out)
        out = self.inception_3c_relu_double_3x3_reduce(out)
        out = self.inception_3c_double_3x3_1(out)
        out = self.inception_3c_double_3x3_1_bn(out)
        out = self.inception_3c_relu_double_3x3_1(out)

        return out


In [0]:
class ECO_2D(nn.Module):
    def __init__(self):
        super(ECO_2D, self).__init__()

        # BasicConvモジュール
        self.basic_conv = BasicConv()

        # Inceptionモジュール
        self.inception_a = InceptionA()
        self.inception_b = InceptionB()
        self.inception_c = InceptionC()

    def forward(self, x):
        '''
        入力xのサイズtorch.Size([batch_num, 3, 224, 224]))
        '''
        out = self.basic_conv(x)
        out = self.inception_a(out)
        out = self.inception_b(out)
        out = self.inception_c(out)

        return out


## 動作確認

In [22]:
# モデルの用意
net = ECO_2D()
net.train()

ECO_2D(
  (basic_conv): BasicConv(
    (conv1_7x7_s2): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (conv1_7x7_s2_bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv1_relu_7x7): ReLU(inplace=True)
    (pool1_3x3_s2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (conv2_3x3_reduce): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
    (conv2_3x3_reduce_bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2_relu_3x3_reduce): ReLU(inplace=True)
    (conv2_3x3): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv2_3x3_bn): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2_relu_3x3): ReLU(inplace=True)
    (pool2_3x3_s2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  )
  (inception_a): InceptionA(
    (inception_3a_1x1): Conv2d(192, 64, kernel_size=(1, 1), 

In [24]:
pip install tensorboardX

     |████████████████████████████████| 204kB 37.8MB/s 


In [0]:
# 1. tensorboardXの保存クラスを呼び出します
from tensorboardX import SummaryWriter

# 2. フォルダ「tbX」に保存させるwriterを用意します
# フォルダ「tbX」はなければ自動で作成されます
writer = SummaryWriter("./tbX/")


# 3. ネットワークに流し込むダミーデータを作成します
batch_size = 1
dummy_img = torch.rand(batch_size, 3, 224, 224)

# 4. netに対して、ダミーデータである
# dummy_imgを流したときのgraphをwriterに保存させます
writer.add_graph(net, (dummy_img, ))
writer.close()


# 5. コマンドプロンプトを開き、フォルダtbXがあるフォルダまで移動して、
# 以下のコマンドを実行します

# tensorboard --logdir="./tbX/"

# その後、http://localhost:6006 にアクセスします


# ECOの3D Netモジュール

In [0]:
class Resnet_3D_3(nn.Module):
    '''Resnet_3D_3'''

    def __init__(self):
        super(Resnet_3D_3, self).__init__()
        
        self.res3a_2 = nn.Conv3d(96, 128, kernel_size=(
            3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
        
        self.res3a_bn = nn.BatchNorm3d(
            128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.res3a_relu = nn.ReLU(inplace=True)

        self.res3b_1 = nn.Conv3d(128, 128, kernel_size=(
            3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
        self.res3b_1_bn = nn.BatchNorm3d(
            128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.res3b_1_relu = nn.ReLU(inplace=True)
        self.res3b_2 = nn.Conv3d(128, 128, kernel_size=(
            3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
        
        self.res3b_bn = nn.BatchNorm3d(
            128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.res3b_relu = nn.ReLU(inplace=True)

    def forward(self, x):

        residual = self.res3a_2(x)
        out = self.res3a_bn(residual)
        out = self.res3a_relu(out)

        out = self.res3b_1(out)
        out = self.res3b_1_bn(out)
        out = self.res3b_relu(out)
        out = self.res3b_2(out)

        out += residual

        out = self.res3b_bn(out)
        out = self.res3b_relu(out)

        return out


In [0]:
class Resnet_3D_4(nn.Module):
    '''Resnet_3D_4'''

    def __init__(self):
        super(Resnet_3D_4, self).__init__()

        self.res4a_1 = nn.Conv3d(128, 256, kernel_size=(
            3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1))
        self.res4a_1_bn = nn.BatchNorm3d(
            256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.res4a_1_relu = nn.ReLU(inplace=True)
        self.res4a_2 = nn.Conv3d(256, 256, kernel_size=(
            3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
        
        self.res4a_down = nn.Conv3d(128, 256, kernel_size=(
            3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1))
        
        self.res4a_bn = nn.BatchNorm3d(
            256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.res4a_relu = nn.ReLU(inplace=True)
        
        self.res4b_1 = nn.Conv3d(256, 256, kernel_size=(
            3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
        self.res4b_1_bn = nn.BatchNorm3d(
            256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.res4b_1_relu = nn.ReLU(inplace=True)
        self.res4b_2 = nn.Conv3d(256, 256, kernel_size=(
            3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
        
        self.res4b_bn = nn.BatchNorm3d(
            256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.res4b_relu = nn.ReLU(inplace=True)

    def forward(self, x):
        residual = self.res4a_down(x)

        out = self.res4a_1(x)
        out = self.res4a_1_bn(out)
        out = self.res4a_1_relu(out)

        out = self.res4a_2(out)

        out += residual

        residual2 = out

        out = self.res4a_bn(out)
        out = self.res4a_relu(out)

        out = self.res4b_1(out)

        out = self.res4b_1_bn(out)
        out = self.res4b_1_relu(out)

        out = self.res4b_2(out)

        out += residual2

        out = self.res4b_bn(out)
        out = self.res4b_relu(out)

        return out


In [0]:
class Resnet_3D_5(nn.Module):
    '''Resnet_3D_5'''

    def __init__(self):
        super(Resnet_3D_5, self).__init__()
        
        self.res5a_1 = nn.Conv3d(256, 512, kernel_size=(
            3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1))
        self.res5a_1_bn = nn.BatchNorm3d(
            512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.res5a_1_relu = nn.ReLU(inplace=True)
        self.res5a_2 = nn.Conv3d(512, 512, kernel_size=(
            3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
        
        self.res5a_down = nn.Conv3d(256, 512, kernel_size=(
            3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1))
        
        self.res5a_bn = nn.BatchNorm3d(
            512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.res5a_relu = nn.ReLU(inplace=True)
        
        self.res5b_1 = nn.Conv3d(512, 512, kernel_size=(
            3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
        self.res5b_1_bn = nn.BatchNorm3d(
            512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.res5b_1_relu = nn.ReLU(inplace=True)
        self.res5b_2 = nn.Conv3d(512, 512, kernel_size=(
            3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
        
        self.res5b_bn = nn.BatchNorm3d(
            512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.res5b_relu = nn.ReLU(inplace=True)

    def forward(self, x):
        residual = self.res5a_down(x)

        out = self.res5a_1(x)
        out = self.res5a_1_bn(out)
        out = self.res5a_1_relu(out)

        out = self.res5a_2(out)

        out += residual  # res5a

        residual2 = out

        out = self.res5a_bn(out)
        out = self.res5a_relu(out)

        out = self.res5b_1(out)

        out = self.res5b_1_bn(out)
        out = self.res5b_1_relu(out)

        out = self.res5b_2(out)

        out += residual2  # res5b

        out = self.res5b_bn(out)
        out = self.res5b_relu(out)

        return out


In [0]:
class ECO_3D(nn.Module):
    def __init__(self):
        super(ECO_3D, self).__init__()

        # 3D_Resnetジュール
        self.res_3d_3 = Resnet_3D_3()
        self.res_3d_4 = Resnet_3D_4()
        self.res_3d_5 = Resnet_3D_5()

        # Global Average Pooling
        self.global_pool = nn.AvgPool3d(
            kernel_size=(4, 7, 7), stride=1, padding=0)

    def forward(self, x):
        '''
        入力xのサイズtorch.Size([batch_num,frames, 96, 28, 28]))
        '''
        out = torch.transpose(x, 1, 2)  # テンソルの順番入れ替え
        out = self.res_3d_3(out)
        out = self.res_3d_4(out)
        out = self.res_3d_5(out)
        out = self.global_pool(out)
        
        # テンソルサイズを変更
        # torch.Size([batch_num, 512, 1, 1, 1])からtorch.Size([batch_num, 512])へ
        out =out.view(out.size()[0], out.size()[1])
        
        return out


## 動作確認

In [30]:
# モデルの用意
net = ECO_3D()
net.train()

ECO_3D(
  (res_3d_3): Resnet_3D_3(
    (res3a_2): Conv3d(96, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (res3a_bn): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (res3a_relu): ReLU(inplace=True)
    (res3b_1): Conv3d(128, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (res3b_1_bn): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (res3b_1_relu): ReLU(inplace=True)
    (res3b_2): Conv3d(128, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (res3b_bn): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (res3b_relu): ReLU(inplace=True)
  )
  (res_3d_4): Resnet_3D_4(
    (res4a_1): Conv3d(128, 256, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1))
    (res4a_1_bn): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (res4a_1_relu): ReLU(inplace=True)
    (res4a_2): Conv3d(2

In [0]:
# 1. tensorboardXの保存クラスを呼び出します
from tensorboardX import SummaryWriter

# 2. フォルダ「tbX」に保存させるwriterを用意します
# フォルダ「tbX」はなければ自動で作成されます
writer = SummaryWriter("./tbX/")


# 3. ネットワークに流し込むダミーデータを作成します
batch_size = 1
dummy_img = torch.rand(batch_size, 16, 96, 28, 28)

# 4. netに対して、ダミーデータである
# dummy_imgを流したときのgraphをwriterに保存させます
writer.add_graph(net, (dummy_img, ))
writer.close()


# 5. コマンドプロンプトを開き、フォルダtbXがあるフォルダまで移動して、
# 以下のコマンドを実行します

# tensorboard --logdir="./tbX/"

# その後、http://localhost:6006 にアクセスします


In [49]:
!git clone https://github.com/activitynet/ActivityNet.git
%cd ActivityNet/Crawler/Kinetics

Cloning into 'ActivityNet'...
remote: Enumerating objects: 253, done.
remote: Total 253 (delta 0), reused 0 (delta 0), pack-reused 253
Receiving objects: 100% (253/253), 44.58 MiB | 30.01 MiB/s, done.
Resolving deltas: 100% (129/129), done.
/content/pytorch_advanced/9_video_classification_eco/ActivityNet/Crawler/Kinetics


In [51]:
!source deactivate

!conda env create -f ./9_video_classification_eco/video_download/environment.yml

/bin/bash: deactivate: No such file or directory
/bin/bash: conda: command not found


In [52]:
pip install --upgrade youtube-dl

     |████████████████████████████████| 1.8MB 38.2MB/s 


In [53]:
pip install --upgrade joblib

Requirement already up-to-date: joblib in /usr/local/lib/python3.6/dist-packages (0.14.1)


In [56]:
%cd  /content/pytorch_advanced/9_video_classification_eco/

/content/pytorch_advanced/9_video_classification_eco


In [57]:
import os

# フォルダ「data」が存在しない場合は作成する
data_dir = "./data/"
if not os.path.exists(data_dir):
    os.mkdir(data_dir)
    
# フォルダ「kinetics_videos」が存在しない場合は作成する
data_dir = "./data/kinetics_videos/"
if not os.path.exists(data_dir):
    os.mkdir(data_dir)
    
    
# フォルダ「video_download」のpytnonファイル「download.py」を実行します
# 取得するyoutubeデータはフォルダ「video_download」のkinetics-400_val_8videos.csvに記載した8動画です
# 保存先はフォルダ「data」内のフォルダ「kinetics_videos」です
!python2 ./video_download/download.py ./video_download/kinetics-400_val_8videos.csv /content/pytorch_advanced/9_video_classification_eco/data/kinetics_videos/

/tmp/kinetics/aab9e9b9-93ac-4faf-a12f-2d0855b18dc3.%(ext)s
/tmp/kinetics/062c6948-122a-4bfe-8b87-e0680a66fd06.%(ext)s
/tmp/kinetics/b8a301b0-3ec4-46f4-b3bd-be7e20e699a7.%(ext)s
/tmp/kinetics/92d95f9c-caec-4097-a3a4-ffa495aab968.%(ext)s
/tmp/kinetics/a5c751e1-a3f0-4620-92bb-a91a8e53c192.%(ext)s
/tmp/kinetics/524b5b7e-3327-4cbb-880f-bafee4d22683.%(ext)s
/tmp/kinetics/e43fc377-46df-4180-8522-aa0122b4b713.%(ext)s
/tmp/kinetics/ccd88db8-f13f-44f9-bb23-456b0330c11c.%(ext)s


# 9.4.2 動画をframeごとに画像データに変換
本ファイルでは、ダウンロードした動画をframeごとにjpeg形式の画像データに変換します。

# 事前準備
ffmepgが入っていない場合には、 以下のコマンドをターミナルで実行し、Ubuntuにてffmpegをインストールします。

sudo apt install ffmpeg

In [58]:
import os
import subprocess  # ターミナルで実行するコマンドを実行できる


# 動画が保存されたフォルダ「kinetics_videos」にある、クラスの種類とパスを取得
dir_path = './data/kinetics_videos'
class_list = os.listdir(path=dir_path)
print(class_list)

# 各クラスの動画ファイルを画像ファイルに変換する
for class_list_i in (class_list):  # クラスごとのループ

    # クラスのフォルダへのパスを取得
    class_path = os.path.join(dir_path, class_list_i)

    # 各クラスのフォルダ内の動画ファイルをひとつずつ処理するループ
    for file_name in os.listdir(class_path):

        # ファイル名と拡張子に分割
        name, ext = os.path.splitext(file_name)

        # mp4ファイルでない、フォルダなどは処理しない
        if ext != '.mp4':
            continue

        # 動画ファイルを画像に分割して保存するフォルダ名を取得
        dst_directory_path = os.path.join(class_path, name)

        # 上記の画像保存フォルダがなければ作成
        if not os.path.exists(dst_directory_path):
            os.mkdir(dst_directory_path)

        # 動画ファイルへのパスを取得
        video_file_path = os.path.join(class_path, file_name)

        # ffmpegを実行させ、動画ファイルをjpgにする （高さは256ピクセルで幅はアスペクト比を変えない）
        # kineticsの動画の場合10秒になっており、大体300ファイルになる（30 frames /sec）
        cmd = 'ffmpeg -i \"{}\" -vf scale=-1:256 \"{}/image_%05d.jpg\"'.format(
            video_file_path, dst_directory_path)
        print(cmd)
        subprocess.call(cmd, shell=True)
        print('\n')

print("動画ファイルを画像ファイルに変換しました。")

['bungee jumping', 'arm wrestling']
ffmpeg -i "./data/kinetics_videos/bungee jumping/TUvSX0pYu4o_000002_000012.mp4" -vf scale=-1:256 "./data/kinetics_videos/bungee jumping/TUvSX0pYu4o_000002_000012/image_%05d.jpg"


ffmpeg -i "./data/kinetics_videos/bungee jumping/dAeUFSdYG1I_000010_000020.mp4" -vf scale=-1:256 "./data/kinetics_videos/bungee jumping/dAeUFSdYG1I_000010_000020/image_%05d.jpg"


ffmpeg -i "./data/kinetics_videos/bungee jumping/b6yQZjPE26c_000023_000033.mp4" -vf scale=-1:256 "./data/kinetics_videos/bungee jumping/b6yQZjPE26c_000023_000033/image_%05d.jpg"


ffmpeg -i "./data/kinetics_videos/bungee jumping/zkXOcxGnUhs_000025_000035.mp4" -vf scale=-1:256 "./data/kinetics_videos/bungee jumping/zkXOcxGnUhs_000025_000035/image_%05d.jpg"


ffmpeg -i "./data/kinetics_videos/arm wrestling/BdMiTo_OtnU_000024_000034.mp4" -vf scale=-1:256 "./data/kinetics_videos/arm wrestling/BdMiTo_OtnU_000024_000034/image_%05d.jpg"


ffmpeg -i "./data/kinetics_videos/arm wrestling/C4lCVBZ3ux0_000028

In [0]:
import os
from PIL import Image
import csv
import numpy as np

import torch
import torch.utils.data
from torch import nn

import torchvision

In [59]:
def make_datapath_list(root_path):
    """
    動画を画像データにしたフォルダへのファイルパスリストを作成する。
    root_path : str、データフォルダへのrootパス
    Returns：ret : video_list、動画を画像データにしたフォルダへのファイルパスリスト
    """

    # 動画を画像データにしたフォルダへのファイルパスリスト
    video_list = list()

    # root_pathにある、クラスの種類とパスを取得
    class_list = os.listdir(path=root_path)

    # 各クラスの動画ファイルを画像化したフォルダへのパスを取得
    for class_list_i in (class_list):  # クラスごとのループ

        # クラスのフォルダへのパスを取得
        class_path = os.path.join(root_path, class_list_i)

        # 各クラスのフォルダ内の画像フォルダを取得するループ
        for file_name in os.listdir(class_path):

            # ファイル名と拡張子に分割
            name, ext = os.path.splitext(file_name)

            # フォルダでないmp4ファイルは無視
            if ext == '.mp4':
                continue

            # 動画ファイルを画像に分割して保存したフォルダのパスを取得
            video_img_directory_path = os.path.join(class_path, name)

            # vieo_listに追加
            video_list.append(video_img_directory_path)

    return video_list


# 動作確認
root_path = './data/kinetics_videos/'
video_list = make_datapath_list(root_path)
print(video_list)
print(video_list[0])
print(video_list[1])

['./data/kinetics_videos/bungee jumping/dAeUFSdYG1I_000010_000020', './data/kinetics_videos/bungee jumping/zkXOcxGnUhs_000025_000035', './data/kinetics_videos/bungee jumping/TUvSX0pYu4o_000002_000012', './data/kinetics_videos/bungee jumping/b6yQZjPE26c_000023_000033', './data/kinetics_videos/arm wrestling/5JzkrOVhPOw_000027_000037', './data/kinetics_videos/arm wrestling/C4lCVBZ3ux0_000028_000038', './data/kinetics_videos/arm wrestling/ehLnj7pXnYE_000027_000037', './data/kinetics_videos/arm wrestling/BdMiTo_OtnU_000024_000034']
./data/kinetics_videos/bungee jumping/dAeUFSdYG1I_000010_000020
./data/kinetics_videos/bungee jumping/zkXOcxGnUhs_000025_000035


In [0]:
import os

import torch
import torch.nn as nn
from torch.nn import init

In [0]:
# フォルダ「weights」が存在しない場合は作成する
weights_dir = "./weights/"
if not os.path.exists(weights_dir):
    os.mkdir(weights_dir)

# フォルダ「weights」に学習済みモデル「ECO_Lite_rgb_model_Kinetics.pth.tar」をダウンロードして配置してください。
# https://github.com/mzolfaghari/ECO-pytorch の
# https://drive.google.com/open?id=1XNIq7byciKgrn011jLBggd2g79jKX4uD

In [67]:
from utils.kinetics400_eco_dataloader import make_datapath_list, VideoTransform, get_label_id_dictionary, VideoDataset

# vieo_listの作成
root_path = './data/kinetics_videos/'
video_list = make_datapath_list(root_path)

# 前処理の設定
resize, crop_size = 224, 224
mean, std = [104, 117, 123], [1, 1, 1]
video_transform = VideoTransform(resize, crop_size, mean, std)

# ラベル辞書の作成
label_dicitionary_path = './video_download/kinetics_400_label_dicitionary.csv'
label_id_dict, id_label_dict = get_label_id_dictionary(label_dicitionary_path)

# Datasetの作成
# num_segments は 動画を何分割して使用するのかを決める
val_dataset = VideoDataset(video_list, label_id_dict, num_segments=16,
                           phase="val", transform=video_transform, img_tmpl='image_{:05d}.jpg')

# DataLoaderにします
batch_size = 8
val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False)

# 動作確認
batch_iterator = iter(val_dataloader)  # イテレータに変換
imgs_transformeds, labels, label_ids, dir_path = next(
    batch_iterator)  # 1番目の要素を取り出す
print(imgs_transformeds.shape)

torch.Size([8, 16, 3, 224, 224])


In [0]:
from utils.eco import ECO_2D, ECO_3D


class ECO_Lite(nn.Module):
    def __init__(self):
        super(ECO_Lite, self).__init__()

        # 2D Netモジュール
        self.eco_2d = ECO_2D()

        # 3D Netモジュール
        self.eco_3d = ECO_3D()

        # クラス分類の全結合層
        self.fc_final = nn.Linear(in_features=512, out_features=400, bias=True)

    def forward(self, x):
        '''
        入力xはtorch.Size([batch_num, num_segments=16, 3, 224, 224]))
        '''

        # 入力xの各次元のサイズを取得する
        bs, ns, c, h, w = x.shape

        # xを(bs*ns, c, h, w)にサイズ変換する
        out = x.view(-1, c, h, w)
        # （注釈）
        # PyTorchのConv2Dは入力のサイズが(batch_num, c, h, w)しか受け付けないため
        # (batch_num, num_segments, c, h, w)は処理できない
        # 今は2次元画像を独立に処理するので、num_segmentsはbatch_numの次元に押し込んでも良いため
        # (batch_num×num_segments, c, h, w)にサイズを変換する

        # 2D Netモジュール 出力torch.Size([batch_num×16, 96, 28, 28])
        out = self.eco_2d(out)

        # 2次元画像をテンソルを3次元用に変換する
        # num_segmentsをbatch_numの次元に押し込んだものを元に戻す
        out = out.view(-1, ns, 96, 28, 28)

        # 3D Netモジュール 出力torch.Size([batch_num, 512])
        out = self.eco_3d(out)

        # クラス分類の全結合層　出力torch.Size([batch_num, class_num=400])
        out = self.fc_final(out)

        return out

In [69]:
net = ECO_Lite()
net

ECO_Lite(
  (eco_2d): ECO_2D(
    (basic_conv): BasicConv(
      (conv1_7x7_s2): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
      (conv1_7x7_s2_bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1_relu_7x7): ReLU(inplace=True)
      (pool1_3x3_s2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
      (conv2_3x3_reduce): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
      (conv2_3x3_reduce_bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2_relu_3x3_reduce): ReLU(inplace=True)
      (conv2_3x3): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2_3x3_bn): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2_relu_3x3): ReLU(inplace=True)
      (pool2_3x3_s2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    )
    (inception_a): InceptionA(
      (inc

In [70]:

# 学習済みモデルをロードする関数の定義


def load_pretrained_ECO(model_dict, pretrained_model_dict):
    '''ECOの学習済みモデルをロードする関数
    今回構築したECOは学習済みモデルとレイヤーの順番は同じだが名前が異なる
    '''

    # 現在のネットワークモデルのパラメータ名
    param_names = []  # パラメータの名前を格納していく
    for name, param in model_dict.items():
        param_names.append(name)

    # 現在のネットワークの情報をコピーして新たなstate_dictを作成
    new_state_dict = model_dict.copy()

    # 新たなstate_dictに学習済みの値を代入
    print("学習済みのパラメータをロードします")
    for index, (key_name, value) in enumerate(pretrained_model_dict.items()):
        name = param_names[index]  # 現在のネットワークでのパラメータ名を取得
        new_state_dict[name] = value  # 値を入れる

        # 何から何にロードされたのかを表示
        print(str(key_name)+"→"+str(name))

    return new_state_dict


# 学習済みモデルをロード
net_model_ECO = "./weights/ECO_Lite_rgb_model_Kinetics.pth.tar"
pretrained_model = torch.load(net_model_ECO, map_location='cpu')
pretrained_model_dict = pretrained_model['state_dict']
# （注釈）
# pthがtarで圧縮されているのは、state_dict以外の情報も一緒に保存されているため。
# そのため読み込むときは辞書型変数になっているので['state_dict']で指定する。

# 現在のモデルの変数名などを取得
model_dict = net.state_dict()

# 学習済みモデルのstate_dictを取得
new_state_dict = load_pretrained_ECO(model_dict, pretrained_model_dict)

# 学習済みモデルのパラメータを代入
net.eval()  # ECOネットワークを推論モードに
net.load_state_dict(new_state_dict)

学習済みのパラメータをロードします
module.base_model.conv1_7x7_s2.weight→eco_2d.basic_conv.conv1_7x7_s2.weight
module.base_model.conv1_7x7_s2.bias→eco_2d.basic_conv.conv1_7x7_s2.bias
module.base_model.conv1_7x7_s2_bn.weight→eco_2d.basic_conv.conv1_7x7_s2_bn.weight
module.base_model.conv1_7x7_s2_bn.bias→eco_2d.basic_conv.conv1_7x7_s2_bn.bias
module.base_model.conv1_7x7_s2_bn.running_mean→eco_2d.basic_conv.conv1_7x7_s2_bn.running_mean
module.base_model.conv1_7x7_s2_bn.running_var→eco_2d.basic_conv.conv1_7x7_s2_bn.running_var
module.base_model.conv1_7x7_s2_bn.num_batches_tracked→eco_2d.basic_conv.conv1_7x7_s2_bn.num_batches_tracked
module.base_model.conv2_3x3_reduce.weight→eco_2d.basic_conv.conv2_3x3_reduce.weight
module.base_model.conv2_3x3_reduce.bias→eco_2d.basic_conv.conv2_3x3_reduce.bias
module.base_model.conv2_3x3_reduce_bn.weight→eco_2d.basic_conv.conv2_3x3_reduce_bn.weight
module.base_model.conv2_3x3_reduce_bn.bias→eco_2d.basic_conv.conv2_3x3_reduce_bn.bias
module.base_model.conv2_3x3_reduce_bn.ru

<All keys matched successfully>

In [71]:
net.eval()  # ECOネットワークを推論モードに

batch_iterator = iter(val_dataloader)  # イテレータに変換
imgs_transformeds, labels, label_ids, dir_path = next(
    batch_iterator)  # 1番目の要素を取り出す

with torch.set_grad_enabled(False):
    outputs = net(imgs_transformeds)  # ECOで推論

print(outputs.shape)  # 出力のサイズ

torch.Size([8, 400])


In [72]:
def show_eco_inference_result(dir_path, outputs_input, id_label_dict, idx=0):
    '''ミニバッチの各データに対して、推論結果の上位を出力する関数を定義'''
    print("ファイル：", dir_path[idx])  # ファイル名

    outputs = outputs_input.clone()  # コピーを作成

    for i in range(5):
        '''1位から5位までを表示'''
        output = outputs[idx]
        _, pred = torch.max(output, dim=0)  # 確率最大値のラベルを予測
        class_idx = int(pred.numpy())  # クラスIDを出力
        print("予測第{}位：{}".format(i+1, id_label_dict[class_idx]))
        outputs[idx][class_idx] = -1000  # 最大値だったものを消す（小さくする）


# 予測を実施
idx = 0
show_eco_inference_result(dir_path, outputs, id_label_dict, idx)

ファイル： ./data/kinetics_videos/bungee jumping/dAeUFSdYG1I_000010_000020
予測第1位：bungee jumping
予測第2位：snowkiting
予測第3位：kitesurfing
予測第4位：parasailing
予測第5位：swinging on something


In [73]:
idx = 4
show_eco_inference_result(dir_path, outputs, id_label_dict, idx)

ファイル： ./data/kinetics_videos/arm wrestling/5JzkrOVhPOw_000027_000037
予測第1位：arm wrestling
予測第2位：stretching leg
予測第3位：headbutting
予測第4位：shaking hands
予測第5位：massaging feet


以上